In [ ]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/23658/1636127/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1604205557&Signature=Rsi2ha3Zfxx6DLhoNEdBSuXzQZ%2BSecoaZiCF53eP8p9IvZGl8aZU%2FpqBo%2BhWgpDtqnHNYYNVzSUZA4magSww2P7lrPW8p1Zljb4D8DdxxgG9gCFY%2FnCT4p1aAajgP%2BDKU55k0XoF2sfrfRBLSlVT9R4pKlQVROl8eaTGHle9J6qDJZbm%2F427HOIy1FxbkhxfuYZqekQoTzP3Pjf5JB%2BLtjtPuuV8q980aYDqscKa3txhtovj9XT%2BfVjd1qqy5ItyCuvzNNTy22KemUirUZWBHtgcG7ULyFvB7gNN4sU2IiTSvO7OL1VJZNYfZlRHiYXToSQPbtq2n1br73spN%2B5Guw%3D%3D&response-content-disposition=attachment%3B+filename%3Ddata-unchained-round-2.zip" -c -O 'data-unchained-round-2.zip'
!unzip 'data-unchained-round-2.zip'

--2020-10-29 11:25:15--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/23658/1636127/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1604205557&Signature=Rsi2ha3Zfxx6DLhoNEdBSuXzQZ%2BSecoaZiCF53eP8p9IvZGl8aZU%2FpqBo%2BhWgpDtqnHNYYNVzSUZA4magSww2P7lrPW8p1Zljb4D8DdxxgG9gCFY%2FnCT4p1aAajgP%2BDKU55k0XoF2sfrfRBLSlVT9R4pKlQVROl8eaTGHle9J6qDJZbm%2F427HOIy1FxbkhxfuYZqekQoTzP3Pjf5JB%2BLtjtPuuV8q980aYDqscKa3txhtovj9XT%2BfVjd1qqy5ItyCuvzNNTy22KemUirUZWBHtgcG7ULyFvB7gNN4sU2IiTSvO7OL1VJZNYfZlRHiYXToSQPbtq2n1br73spN%2B5Guw%3D%3D&response-content-disposition=attachment%3B+filename%3Ddata-unchained-round-2.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.7.240, 172.217.9.208, 172.217.164.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.7.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1719276 (1.6M) [application/zip]
Saving to: ‘data-unchained-round-2.zip’

d

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from sklearn.decomposition import *

!pip install catboost
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

     |████████████████████████████████| 66.2MB 45kB/s 


In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [ ]:
train.head()

,id,content,type
0,1694628490,@avataraang awww ) where'd you get that? hugh ...,surprise
1,1751142947,@mistressmia Their is no such thing as TRANSPA...,enthusiasm
2,1695004607,"@Dojie wouldn't that cost quite a bit, I mean ...",fun
3,1752918270,Eminem's new song &quot;Beautiful&quot; is ama...,surprise
4,1751176554,"@AmyriadfthINGs Yeah, I've never seen that! Bu...",surprise


#*EDA AND FEATURE ENGINEERING*

In [ ]:
!pip install sentence_transformers
import nltk
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 1.1MB 8.2MB/s 
     |████████████████████████████████| 1.1MB 51.4MB/s 
     |████████████████████████████████| 3.0MB 51.7MB/s 
     |████████████████████████████████| 890kB 49.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=5cae87781b07edffc31b76606cde276da51786973c7629095f7643b23144e091
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=543265da642554c9aae6f56ca26faf83b41f8fe9397c71fa74cb2e6639f2b15f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
full_df = pd.concat([train, test]).reset_index(drop=True)

In [ ]:
full_df['type'].unique()

array(['surprise', 'enthusiasm', 'fun', 'love', 'sadness', 'neutral',
       'worry', 'hate', 'happiness', 'anger', 'empty', 'relief',
       'boredom', nan], dtype=object)

In [ ]:
full_df['type'].value_counts()

neutral       6484
worry         6398
happiness     3896
sadness       3874
love          2876
surprise      1656
fun           1332
relief        1118
hate           975
empty          633
enthusiasm     542
boredom        132
anger           84
Name: type, dtype: int64

In [ ]:
import re
def find_links(content):
    link = re.findall(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', content)
    if link == []:
        return 0
    else:
        return 1
full_df['is_link'] = full_df['content'].apply(find_links)
full_df['length'] = full_df['content'].apply(len)

In [ ]:
word_embeddings = SentenceTransformer('roberta-base-nli-mean-tokens').encode(full_df.content)
word_embeddings = pd.DataFrame(data=word_embeddings)

100%|██████████| 460M/460M [00:26<00:00, 17.5MB/s]


In [ ]:
word_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.936990,-0.680105,-0.159944,-0.900732,-0.905331,0.443983,1.051530,0.767195,-0.247284,0.732031,0.340042,-0.250482,-0.045357,-0.004228,0.886420,0.127953,-2.213039,-0.153034,-1.024973,0.466459,0.324650,0.656358,-0.375432,0.757913,0.998230,0.311717,0.512705,0.970552,-0.180337,-0.350281,-0.853180,0.349862,0.316250,-0.043303,-0.066526,0.276586,1.115276,-0.843326,0.796359,-0.613143,...,-0.833989,0.838041,-0.552931,-1.416171,0.614420,-0.218332,-0.379649,0.310769,0.886570,-0.535731,-0.417061,0.770466,-0.317296,-0.728205,1.287679,0.043256,0.208637,0.011052,0.755853,-0.112164,-0.019650,-1.127185,0.220656,0.256585,-1.469070,-0.714864,-0.005237,0.290377,0.603744,1.316305,0.382246,0.575600,-1.146771,-0.655219,0.149285,0.564739,-0.503086,-0.590026,0.024991,0.218836
1,-0.019380,-0.156352,-0.137031,-1.286854,0.368093,0.979236,1.029281,0.074945,-0.295842,1.048326,-0.273815,0.287886,-0.148186,0.359399,0.212254,-1.879352,-1.959218,0.546132,-1.260391,0.361085,0.251574,1.671937,-1.747775,0.766887,0.684744,-0.101421,0.168502,0.668032,-0.526166,-0.251606,-0.499979,-0.255432,-0.266678,0.344049,-0.593885,0.384818,0.407690,-0.928294,0.625413,-0.531066,...,-1.100904,-0.247308,-0.407300,-0.597838,0.464952,0.900612,-0.027438,0.347143,1.236196,-0.625879,-0.751445,-0.666308,-0.284490,0.014226,0.989707,0.252623,-0.283965,0.951479,-0.061939,-0.737634,-0.398582,-1.769308,-0.436461,-0.320245,-1.116614,-0.314068,-0.287800,0.075478,0.793709,0.982331,0.281877,0.340823,0.726070,-0.272946,-0.223233,0.390455,-0.512701,-1.270007,-0.082249,-0.413046
2,-0.149273,0.021564,-0.079297,0.187080,0.479027,-0.103631,0.295510,-0.634702,-0.826587,0.198417,0.252302,-1.217972,0.666460,0.669447,0.916991,0.788673,-0.575538,-0.255187,0.268818,-0.692054,-0.718655,1.235490,1.348150,0.371605,0.397554,-0.387761,1.175558,-0.394170,-0.630226,-1.193009,-0.924076,0.522190,-0.252105,0.334005,0.298769,0.899546,1.303390,-0.556447,-0.314724,-0.755651,...,0.242193,-0.785418,0.165865,-1.234828,-1.393995,0.552770,1.076048,0.631240,0.016355,-0.218908,0.174996,1.562008,-1.569131,-0.619263,1.138046,0.174625,0.397854,0.589965,-1.124891,0.004337,0.644015,-0.391225,-0.421571,-1.172906,-1.966286,0.170438,-0.350570,-0.577970,0.868156,0.211366,-0.127292,1.476040,1.052209,-0.400107,0.285238,0.284532,-0.209430,-1.265322,-0.441447,1.015999
3,0.672211,-0.476154,-0.542591,-0.182478,-0.179125,0.373330,0.725957,-0.903435,-0.543745,-0.014312,0.269214,1.387714,-0.044598,1.464166,0.922190,0.705639,-0.930065,-0.022808,-1.444357,-0.690002,0.595173,-0.487583,0.412010,0.565773,1.029765,-0.384394,0.783079,-0.282899,-0.777045,0.390207,-1.272156,0.355578,0.491726,-0.989031,-0.301703,0.670105,1.083099,-1.330192,0.936027,-0.427076,...,0.215925,0.464788,-0.303616,-1.675628,-0.121619,-1.178732,0.668099,1.292268,-0.068238,0.079421,-0.919850,0.076353,-0.729649,-0.467155,1.503947,-0.503463,0.516594,-0.027132,-0.996343,0.948649,-0.642599,0.067391,2.810165,-0.150351,-2.287706,0.198910,-0.474237,0.612414,0.358550,0.209696,-1.038454,0.521083,-0.707298,-0.268342,0.571078,0.641324,0.571280,-2.004977,0.034388,0.996234
4,0.773176,-0.761485,-0.559050,-1.187123,-0.509251,1.661998,0.543909,0.213183,-1.084707,0.764962,0.037339,0.978758,-0.453137,0.227820,0.188103,-0.953647,-2.318840,0.036265,-1.899488,0.131920,0.850323,0.630671,-1.162809,0.840146,0.785801,-0.194131,0.382612,1.710492,0.199281,-0.224469,-0.651323,-0.056605,0.158007,-0.027424,-0.636696,0.005917,0.595803,-0.204033,0.338188,-0.529189,...,-1.557830,0.445924,-0.718465,-1.880746,0.951727,-1.061726,-0.248046,-0.250730,0.971551,-0.320119,-0.886348,0.516958,-1.217891,0.428047,0.779566,-0.826002,0.354475,0.274407,0.494110,0.283919,0.094111,-0.596367,0.103744,0.078099,-1.529846,-0.834567,0.285045,-0.031158,1.481976,1.

In [ ]:
target_map = {key:value for value,key in enumerate(train['type'].unique())}
target_rev_map = {key:value for key,value in enumerate(train['type'].unique())}
full_df['type'] = full_df['type'].map(target_map)

full_df = pd.concat([full_df, word_embeddings], axis=1)
full_df

,id,content,type,is_link,length,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,1694628490,@avataraang awww ) where'd you get that? hugh ...,0.0,0,130,0.936990,-0.680105,-0.159944,-0.900732,-0.905331,0.443983,1.051530,0.767195,-0.247284,0.732031,0.340042,-0.250482,-0.045357,-0.004228,0.886420,0.127953,-2.213039,-0.153034,-1.024973,0.466459,0.324650,0.656358,-0.375432,0.757913,0.998230,0.311717,0.512705,0.970552,-0.180337,-0.350281,-0.853180,0.349862,0.316250,-0.043303,-0.066526,...,-0.833989,0.838041,-0.552931,-1.416171,0.614420,-0.218332,-0.379649,0.310769,0.886570,-0.535731,-0.417061,0.770466,-0.317296,-0.728205,1.287679,0.043256,0.208637,0.011052,0.755853,-0.112164,-0.019650,-1.127185,0.220656,0.256585,-1.469070,-0.714864,-0.005237,0.290377,0.603744,1.316305,0.382246,0.575600,-1.146771,-0.655219,0.149285,0.564739,-0.503086,-0.590026,0.024991,0.218836
1,1751142947,@mistressmia Their is no such thing as TRANSPA...,1.0,0,77,-0.019380,-0.156352,-0.137031,-1.286854,0.368093,0.979236,1.029281,0.074945,-0.295842,1.048326,-0.273815,0.287886,-0.148186,0.359399,0.212254,-1.879352,-1.959218,0.546132,-1.260391,0.361085,0.251574,1.671937,-1.747775,0.766887,0.684744,-0.101421,0.168502,0.668032,-0.526166,-0.251606,-0.499979,-0.255432,-0.266678,0.344049,-0.593885,...,-1.100904,-0.247308,-0.407300,-0.597838,0.464952,0.900612,-0.027438,0.347143,1.236196,-0.625879,-0.751445,-0.666308,-0.284490,0.014226,0.989707,0.252623,-0.283965,0.951479,-0.061939,-0.737634,-0.398582,-1.769308,-0.436461,-0.320245,-1.116614,-0.314068,-0.287800,0.075478,0.793709,0.982331,0.281877,0.340823,0.726070,-0.272946,-0.223233,0.390455,-0.512701,-1.270007,-0.082249,-0.413046
2,1695004607,"@Dojie wouldn't that cost quite a bit, I mean ...",2.0,0,112,-0.149273,0.021564,-0.079297,0.187080,0.479027,-0.103631,0.295510,-0.634702,-0.826587,0.198417,0.252302,-1.217972,0.666460,0.669447,0.916991,0.788673,-0.575538,-0.255187,0.268818,-0.692054,-0.718655,1.235490,1.348150,0.371605,0.397554,-0.387761,1.175558,-0.394170,-0.630226,-1.193009,-0.924076,0.522190,-0.252105,0.334005,0.298769,...,0.242193,-0.785418,0.165865,-1.234828,-1.393995,0.552770,1.076048,0.631240,0.016355,-0.218908,0.174996,1.562008,-1.569131,-0.619263,1.138046,0.174625,0.397854,0.589965,-1.124891,0.004337,0.644015,-0.391225,-0.421571,-1.172906,-1.966286,0.170438,-0.350570,-0.577970,0.868156,0.211366,-0.127292,1.476040,1.052209,-0.400107,0.285238,0.284532,-0.209430,-1.265322,-0.441447,1.015999
3,1752918270,Eminem's new song &quot;Beautiful&quot; is ama...,0.0,1,117,0.672211,-0.476154,-0.542591,-0.182478,-0.179125,0.373330,0.725957,-0.903435,-0.543745,-0.014312,0.269214,1.387714,-0.044598,1.464166,0.922190,0.705639,-0.930065,-0.022808,-1.444357,-0.690002,0.595173,-0.487583,0.412010,0.565773,1.029765,-0.384394,0.783079,-0.282899,-0.777045,0.390207,-1.272156,0.355578,0.491726,-0.989031,-0.301703,...,0.215925,0.464788,-0.303616,-1.675628,-0.121619,-1.178732,0.668099,1.292268,-0.068238,0.079421,-0.919850,0.076353,-0.729649,-0.467155,1.503947,-0.503463,0.516594,-0.027132,-0.996343,0.948649,-0.642599,0.067391,2.810165,-0.150351,-2.287706,0.198910,-0.474237,0.612414,0.358550,0.209696,-1.038454,0.521083,-0.707298,-0.268342,0.571078,0.641324,0.571280,-2.004977,0.034388,0.996234
4,1751176554,"@AmyriadfthINGs Yeah, I've never seen that! Bu...",0.0,0,85,0.773176,-0.761485,-0.559050,-1.187123,-0.509251,1.661998,0.543909,0.213183,-1.084707,0.764962,0.037339,0.978758,-0.453137,0.227820,0.188103,-0.953647,-2.318840,0.036265,-1.899488,0.131920,0.850323,0.630671,-1.162809,0.840146,0.785801,-0.194131,0.382612,1.710492,0.199281,-0.224469,-0.651323,-0.056605,0.158007,-0.027424,-0.636696,...,-1.557830,0.445924,-0.718465,-1.880746,0.951727,-1.061726,-0.248046,-0.250730,0.971551,-0.320119,-0.886348,0.516958,-1.217891,0.428047,0.779

#*MAKING X AND Y*

In [ ]:
train_df = full_df[full_df['type'].notna()]
test_df = full_df[full_df['type'].isna()]

In [ ]:
X = train_df.drop(['type', 'content', 'id'], axis=1)
y = train_df['type'].astype(int)

#*MODEL*

In [ ]:
skfold = StratifiedKFold(5, shuffle=True, random_state=1235)

predictions_cat = []
accuracies_cat = []
probabilities_cat = []

final_predictions = []
final_accuracies = []
final_probabilities = []

skfold.get_n_splits(X,y)

fold = 1

for train_idx, test_idx in skfold.split(X,y):

    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

    print(f'FOLD: {fold}')

    # dropped_columns = []

    model_cat = CatBoostClassifier(od_type='iter', iterations=5000, task_type='GPU', eval_metric='Accuracy')
    model_cat.fit(X_train.drop(dropped_columns, axis=1), y_train,
                  eval_set = (X_test.drop(dropped_columns, axis=1), y_test),
                #   cat_features = ['is_link'],
                  early_stopping_rounds = 300,
                  verbose = 300)
    labels_cat = model_cat.predict(X_test.drop(dropped_columns, axis=1))
    score_cat = accuracy_score(y_test, labels_cat)
    prob_cat = model_cat.predict_proba(X_test.drop(dropped_columns, axis=1))
    print(f'CatBoost Score: {score_cat}')
    accuracies_cat.append(score_cat)

    prediction_cat = model_cat.predict(test_df[X_train.drop(dropped_columns, axis=1).columns])
    prediction_cat = pd.DataFrame(data = prediction_cat, index=test_df.id, columns=['type'])
    prediction_cat['type'] = prediction_cat['type'].map(target_rev_map)
    predictions_cat.append(prediction_cat)
    probabilities_cat.append(pd.DataFrame(model_cat.predict_proba(test_df[X_train.drop(dropped_columns, axis=1).columns])))

    print('--'*60)
    fold += 1

print()
print(f'MAX: {max(accuracies_cat)}')
print(f'MIN: {min(accuracies_cat)}')
print(f'MEAN: {np.mean(accuracies_cat)}')

FOLD: 1
Learning rate set to 0.071744
0:	learn: 0.3343333	test: 0.3225000	best: 0.3225000 (0)	total: 81.1ms	remaining: 6m 45s
300:	learn: 0.4800833	test: 0.3653333	best: 0.3655000 (299)	total: 11.1s	remaining: 2m 53s
600:	learn: 0.5787500	test: 0.3646667	best: 0.3668333 (458)	total: 21.8s	remaining: 2m 39s
bestTest = 0.3668333333
bestIteration = 458
Shrink model to first 459 iterations.
CatBoost Score: 0.36683333333333334
------------------------------------------------------------------------------------------------------------------------
FOLD: 2
Learning rate set to 0.071744
0:	learn: 0.3255000	test: 0.3225000	best: 0.3225000 (0)	total: 46.7ms	remaining: 3m 53s
300:	learn: 0.4744583	test: 0.3795000	best: 0.3795000 (300)	total: 11.1s	remaining: 2m 53s
600:	learn: 0.5732083	test: 0.3818333	best: 0.3831667 (596)	total: 21.8s	remaining: 2m 39s
bestTest = 0.3831666667
bestIteration = 596
Shrink model to first 597 iterations.
CatBoost Score: 0.38316666666666666
---------------------------

In [ ]:
imp = pd.DataFrame(data=model_cat.feature_importances_, index=X_train.drop(dropped_columns, axis=1).columns, columns=['imp']).sort_values(by='imp', ascending=False)
dropped_columns = imp[imp['imp'] == 0].index

In [ ]:
ens_prob = sum(probabilities_cat)/5
id = test_df.id
ens_labels = []
for row in ens_prob.iterrows():
    val = list(row[1].values)
    idx = val.index(max(val))
    ens_labels.append(idx)
ens_predictions = pd.DataFrame(data=ens_labels, index=id, columns=['type'])
ens_predictions['type'] = ens_predictions['type'].map(target_rev_map)

In [ ]:
ens_predictions.to_csv('submission.csv')